In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# specify substep parameters for interactive run
# this cell will be replaced during job run with the parameters from json within params subfolder
substep_params={
    "mae_threshold":1.5 ,
    "mape_threshold":0.8}

In [3]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

**Pipeline params:**


{'X': 'something',
 'env_name': 'user',
 'pipeline_name': 'pipeline',
 'zone_name': 'zone'}




**Step params:**


{'Y': 'something_else'}




In [4]:
#3 define substep interface
from sinara.substep import NotebookSubstep, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface(
   
    inputs =
    [
        { STEP_NAME: "model_train", ENTITY_NAME: "california_bento" }, #, ZONE_NAME: "zone" },
        
    ],
    outputs = 
    [
       
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

'tmp' dir is not valid, creating valid tmp dir...


**STEP NAME:**


'model_eval'




**INPUTS:**


[{'user.pipeline.zone.model_train.california_bento': '/data/home/jovyan/pipeline/zone/model_train/run-25-01-15-070811/california_bento'}]




In [5]:
#4 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

Session is run


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/15 08:22:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/15 08:22:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:

substep.add_metric('experiment_name', 'evaluation_metrics_example')

In [7]:
# read inputs 
bento_step_inputs = substep.inputs(step_name="model_train")

In [9]:

# load a bentoservice
from sinara.bentoml import load_bentoservice
from sklearn.metrics import mean_absolute_percentage_error,mean_absolute_error
bento_serv = load_bentoservice(bento_step_inputs.california_bento)

[2025-01-15 08:24:57,840] WARNING - Using BentoML not from official PyPI release. In order to find the same version of BentoML when deploying your BentoService, you must set the 'core/bentoml_deploy_version' config to a http/git location of your BentoML fork, e.g.: 'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2025-01-15 08:24:57,868] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.13.2, but loading from BentoML version 0.13.2+6.g2dc5913c
[2025-01-15 08:24:58,049] WARNING - Python 3.10.12 found in current environment is not officially supported by BentoML. The docker base image used is'bentoml/model-server:0.13.2' which will use conda to install Python 3.10.12 in the build process. Supported Python versions are: f3.6, 3.7, 3.8


In [10]:
bento_service_version = bento_serv.service_version().split('.')[-1]
substep.add_metric("version", 
                   {'bento_service_version': bento_service_version})
print(bento_service_version)

run-25-01-15-070811


In [13]:
import pandas as pd
test_data = bento_serv.test_data()
predictions = bento_serv.predict(pd.DataFrame(test_data['X']))
mae = mean_absolute_error(pd.DataFrame(test_data['Y']).values, predictions)
mape = mean_absolute_percentage_error(pd.DataFrame(test_data['Y']).values, predictions)

print("The mean_absolute_error (MAE) on test set: {:.4f}".format(mae))
print("The mean_absolute_percentage_errorr (MAPE) on test set: {:.4f}".format(mape))

The mean_absolute_error (MAE) on test set: 0.9240
The mean_absolute_percentage_errorr (MAPE) on test set: 0.6169


In [14]:
# add evaluation results to the step metrics
substep.add_metric('eval_result', {'mae': mae,
                              'mape': mape})

In [15]:
# check eval result
mae_threshold = substep_params["mae_threshold"]
mape_threshold = substep_params["mape_threshold"]
if mae > mae_threshold:
    raise Exception(f'MAE is {mae}, more than acceptable value of {mae_threshold}')
if mape > mape_threshold:
    raise Exception(f'MAPE is {mape}, more than acceptable value of {mape_threshold}')